## Statistic of random walk-generated graph context

In my master studies, I am interested in complex networks and their building blocks - motifs. Inspired by the skipgram NLP model and the adaptation to network DeepWalk, I think that if we use network motifs as a biased random walk pattern, we can generate a more expressive graph context in term of mesoscopic structure emphasis.

In [2]:
from sys import path
path.append('./../src/')  # import prototype modules
from constrains import R, UTriangle, UWedge
from walks import WalkGenerator
import networkx as nx
import numpy as np
import sklearn as sk
from matplotlib import pyplot as plt
import pickle
%matplotlib inline

data_root = './../data/'

## Amazon co-purchase network

In [3]:
amazon = pickle.load(open(data_root+'amazon.graph', 'rb'))
assert isinstance(amazon, nx.Graph)
assert amazon.size() == 925872

Random walk context (identical to DeepWalk) is generated by WalkGenerator with different constrains.

In [4]:
random_walker = WalkGenerator(graph=amazon, constrain=R())
amazon_random_context = []

We want to create a random context, each walk length is 80, there is 20000 walks in total.

In [13]:
amazon_random_context = [i[:] for i in random_walker._gen(num_walk=20000, walk_length=80)]

In [14]:
from collections import defaultdict as dd

amazon_random_node_count = dd(int)
for node_list in amazon_random_context:
    for i in node_list:
        amazon_random_node_count[i] += 1

In [15]:
len(amazon_random_node_count)

263385

In [17]:
max(amazon_random_node_count, key=lambda i: amazon_random_node_count[i])

110284

In [18]:
amazon_random_node_count[110284]

622